In [83]:
import numpy as np
import ccdm
import numpy as np
from functools import reduce
import itertools

In [2]:
def bound_interval_binary(seq, P_X, k):
    # seq: sequência como lista de '0' e '1'
    # P_X: dicionário com probabilidades
    # k: número de bits de entrada

    Pn = np.prod([P_X[s] for s in seq])
    ratio = 2**(-k) / Pn
    upper_bound = 1 / (P_X['0'] * P_X['1'])

    return {
        "lower_bound": 1,
        "ratio": ratio,
        "upper_bound": upper_bound
    }

def bound_interval_nonbinary(seq, P_X, k):
    Pn = np.prod([P_X[s] for s in seq])
    ratio = 2**(-k) / Pn

    p_min = min(P_X.values())
    p_max = max(P_X.values())
    upper_bound = 1 / (p_min * p_max)

    return {
        "lower_bound": 1,
        "ratio": ratio,
        "upper_bound": upper_bound
    }


def kl_upper_bound_binary(P_X):
    return np.log2(1 / (P_X['0'] * P_X['1']))

def kl_upper_bound_nonbinary(P_X):
    p_min = min(P_X.values())
    p_max = max(P_X.values())
    return -np.log2(p_min * p_max)

def prob_sequence(seq, P_X):
    return np.prod([P_X[s] for s in seq])


In [3]:
P_X = {'0': 0.4, '1': 0.6}
seq = ['1', '0', '1', '1']
k = 3  # bits de entrada

print(bound_interval_binary(seq, P_X, k))
print(kl_upper_bound_binary(P_X))
print("P(a^n) =", prob_sequence(seq, P_X))  # Output: 0.147

{'lower_bound': 1, 'ratio': 1.4467592592592595, 'upper_bound': 4.166666666666667}
2.0588936890535687
P(a^n) = 0.08639999999999999


In [140]:
def cumulative_prob(P):
    """Cria os limites cumulativos para cada símbolo"""
    cum = {}
    total = 0.0
    for symbol, prob in sorted(P.items()):
        cum[symbol] = (total, total + prob)
        total += prob
    return cum

def matching_canditade_intervals(symbol, source_interval, cum):
    """Encontra os intervalos de símbolos que coincidem com os da fonte"""
    matching = {}
    symbol_low_source, symbol_high_source = source_interval
    for cc in cum:
        symbol_low_canditate, symbol_high_canditate = cum[cc]
        if symbol_low_source >= symbol_low_canditate and symbol_high_source <= symbol_high_canditate:
            return symbol, symbol_low_canditate, symbol_high_canditate
    return None, None, None

def matching_canditade_intervals_finalize(symbol, source_interval, cum):
    """Encontra os intervalos de símbolos que coincidem com os da fonte"""
    matching = {}
    high = []
    low = []
    symbol_low_source, symbol_high_source = source_interval
    for cc in cum:
        symbol_low_canditate, symbol_high_canditate = cum[cc]
        if symbol_low_source >= symbol_low_canditate:
            matching = {cc: (symbol_low_canditate, symbol_high_canditate)}
            low.append(matching)
        if symbol_high_source <= symbol_high_canditate:
            matching = {cc: (symbol_low_canditate, symbol_high_canditate)}
            high.append(matching)
    return max(low, key=lambda d: list(d.values())[0][0]) | min(high, key=lambda d: list(d.values())[0][0])

def rescale_source_interval(P_source, source_interval, new_low, new_high):
    """Redimensiona os intervalos cumulativos para corresponder aos da fonte"""
    
    low, high = source_interval
    range_ = new_high - new_low
    new_bounder_low = (low - new_low)/ range_
    new_bounder_high = (high - new_low) / range_
    mid = (new_bounder_low + new_bounder_high) / 2
    P_source = {0: (new_bounder_low, mid), 1: (mid, new_bounder_high)}
    return P_source

def rescale_source_code(P_code, new_low, new_high, P=0.5):
    """Redimensiona os intervalos cumulativos para corresponder aos da fonte"""  
    range_ = new_high - new_low
    mid = (new_low + P * range_)
    P_code = {0: (new_low, mid), 1: (mid, new_high)}
    return P_code

def update_code_interval(code_intervals, P):
    _, m1 = code_intervals[0]
    new_code_intervals = []
    for symbol in code_intervals:
        lowerBound, upperBound  = code_intervals[symbol]
        probability = P[symbol]
        if symbol == 0:
            upperBound = m1 - (1-probability) * (m1 - lowerBound)
            new_code_intervals.append((lowerBound, upperBound))
            new_code_intervals.append((upperBound, m1))
        else:
            lowerBound = m1 + (1-probability) * (upperBound - m1)
            new_code_intervals.append((m1, lowerBound))
            new_code_intervals.append((lowerBound, upperBound))
    return new_code_intervals

def arithmetic_encode(sequence, P, P_source):
    """Codifica uma sequência usando codificação aritmética"""
    cum = cumulative_prob(P)
    cum_source = cumulative_prob(P_source)
    low = 0.0
    high = 1.0
    out_bits = 0
    out = []
    print("Intervalo Fonte:", cum_source)
    print("Intervalo Candidato:", cum)
    

    for symbol in sequence:
        print("\n--> Símbolo em teste", symbol)
        source_interval = cum_source[symbol]
        new_sym, new_low, new_high = matching_canditade_intervals(symbol, source_interval, cum)
        if new_sym:
            cum_source = rescale_source_interval(cum_source, source_interval, new_low, new_high)
            print("\nIntervalo encontrado")
            print("Intervalo Fonte:", cum_source)
            print("Intervalo Candidato:", cum)
            out.extend([new_sym])
            out_bits+=1
        
        else: 
            new_interval = update_code_interval(cum, P)
            # new_code_interval = {i: intervalo for i, intervalo in enumerate(x)}
            n_bits = np.log2(len(new_interval)).astype(int)
            # Gera as chaves binárias (tuplas de 0 e 1)
            chaves = list(itertools.product([0, 1], repeat=n_bits))
            # Cria o dicionário
            new_code_interval = {chave: intervalo for chave, intervalo in zip(chaves, new_interval)}
            print("\nIntervalo Fonte:", cum_source)
            print("Intervalo Candidato:", new_code_interval)
            

            new_sym, new_low, new_high = matching_canditade_intervals(symbol, source_interval, new_code_interval)
            if new_sym:
                cum_source = rescale_source_interval(cum_source, source_interval, new_low, new_high)
                print("\nIntervalo encontrado")
                print("Intervalo Fonte:", cum_source)
                print("Intervalo Candidato:", new_code_interval)
                
                out.extend([new_sym])
                out_bits+=1
            else:
                new_code_interval = matching_canditade_intervals_finalize(symbol, source_interval, new_code_interval)
                print("aqui", new_code_interval)
                out_bits+=1
                subdivididos = {}
                for chave, (inicio, fim) in new_code_interval.items():
                    comprimento = fim - inicio
                    ponto_meio = inicio + 0.4 * comprimento  # 40% do intervalo
                    
                    subdivididos[(chave, 0)] = (inicio, ponto_meio)
                    subdivididos[(chave, 1)] = (ponto_meio, fim)
   
                fim = []
                for chave, (low_code, high_code) in subdivididos.items():
                    
                    low_source, high_source = source_interval
                    if low_source < low_code and high_source > high_code:
                        fim.append((chave, (low_code, high_code)))

                print(fim)
    print("CC:", out)

    # (low, high) = P_source[new_sym]
    # range_ = high - low
    # high = low + range_ * symbol_high
    # low = low + range_ * symbol_low
    # print(f"Símbolo: {symbol}, Intervalo: [{low:.10f}, {high:.10f})")

    # O valor final pode ser qualquer número entre [low, high)
    # Usamos o ponto médio como representação
    
    return None



In [141]:
# Exemplo
P = {0: 0.4, 1: 0.6}
P_source = {0: 0.5, 1: 0.5}
sequence = [1, 0]

matching = arithmetic_encode(sequence, P, P_source)


Intervalo Fonte: {0: (0.0, 0.5), 1: (0.5, 1.0)}
Intervalo Candidato: {0: (0.0, 0.4), 1: (0.4, 1.0)}

--> Símbolo em teste 1

Intervalo encontrado
Intervalo Fonte: {0: (0.16666666666666663, 0.5833333333333333), 1: (0.5833333333333333, 1.0)}
Intervalo Candidato: {0: (0.0, 0.4), 1: (0.4, 1.0)}

--> Símbolo em teste 0

Intervalo Fonte: {0: (0.16666666666666663, 0.5833333333333333), 1: (0.5833333333333333, 1.0)}
Intervalo Candidato: {(0, 0): (0.0, 0.16000000000000003), (0, 1): (0.16000000000000003, 0.4), (1, 0): (0.4, 0.64), (1, 1): (0.64, 1.0)}
aqui {(0, 1): (0.16000000000000003, 0.4), (1, 0): (0.4, 0.64)}
[(((0, 1), 1), (0.256, 0.4)), (((1, 0), 0), (0.4, 0.496))]
CC: [1]


In [82]:
import itertools

intervalos = [
    (0.0, 0.16000000000000003),
    (0.16000000000000003, 0.4),
    (0.4, 0.64),
    (0.64, 1.0)
]

# Número de bits para representar 4 combinações
n_bits = np.log2(len(intervalos)).astype(int)

# Gera as chaves binárias (tuplas de 0 e 1)
chaves = list(itertools.product([0, 1], repeat=n_bits))

# Cria o dicionário
dicionario = {chave: intervalo for chave, intervalo in zip(chaves, intervalos)}

# Visualização
for k, v in dicionario.items():
    print(f"{k}: {v}")


(0, 0): (0.0, 0.16000000000000003)
(0, 1): (0.16000000000000003, 0.4)
(1, 0): (0.4, 0.64)
(1, 1): (0.64, 1.0)
